Για να εκτελέσετε τα παρακάτω notebooks, αν δεν το έχετε κάνει ήδη, πρέπει να ορίσετε το openai key μέσα στο αρχείο .env ως `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Στη συνέχεια, θα φορτώσουμε το Embedding Index σε ένα Pandas Dataframe. Το Embedding Index είναι αποθηκευμένο σε ένα αρχείο JSON με όνομα `embedding_index_3m.json`. Το Embedding Index περιέχει τα Embeddings για κάθε ένα από τα απομαγνητοφωνημένα κείμενα του YouTube μέχρι τα τέλη Οκτωβρίου 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Στη συνέχεια, θα δημιουργήσουμε μια συνάρτηση με όνομα `get_videos` που θα αναζητά το Embedding Index για το ερώτημα. Η συνάρτηση θα επιστρέφει τα 5 βίντεο που μοιάζουν περισσότερο με το ερώτημα. Η συνάρτηση λειτουργεί ως εξής:

1. Πρώτα, δημιουργείται ένα αντίγραφο του Embedding Index.
2. Έπειτα, υπολογίζεται το Embedding για το ερώτημα χρησιμοποιώντας το OpenAI Embedding API.
3. Στη συνέχεια, δημιουργείται μια νέα στήλη στο Embedding Index με όνομα `similarity`. Η στήλη `similarity` περιέχει την συνημίτονο ομοιότητα (cosine similarity) μεταξύ του Embedding του ερωτήματος και του Embedding για κάθε τμήμα βίντεο.
4. Έπειτα, το Embedding Index φιλτράρεται με βάση τη στήλη `similarity`. Το Embedding Index φιλτράρεται ώστε να περιλαμβάνει μόνο βίντεο που έχουν συνημίτονο ομοιότητα μεγαλύτερη ή ίση με 0.75.
5. Τέλος, το Embedding Index ταξινομείται με βάση τη στήλη `similarity` και επιστρέφονται τα 5 κορυφαία βίντεο.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Αυτή η συνάρτηση είναι πολύ απλή, απλώς εκτυπώνει τα αποτελέσματα του ερωτήματος αναζήτησης.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Πρώτα, το Embedding Index φορτώνεται σε ένα Pandas Dataframe.
2. Στη συνέχεια, ζητείται από τον χρήστη να εισάγει ένα ερώτημα.
3. Έπειτα, καλείται η συνάρτηση `get_videos` για να αναζητήσει το ερώτημα στο Embedding Index.
4. Τέλος, καλείται η συνάρτηση `display_results` για να εμφανίσει τα αποτελέσματα στον χρήστη.
5. Στη συνέχεια, ζητείται από τον χρήστη να εισάγει ένα νέο ερώτημα. Αυτή η διαδικασία συνεχίζεται μέχρι ο χρήστης να πληκτρολογήσει `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.el.png)

Θα σας ζητηθεί να εισάγετε ένα ερώτημα. Πληκτρολογήστε το ερώτημά σας και πατήστε enter. Η εφαρμογή θα επιστρέψει μια λίστα με βίντεο που σχετίζονται με το ερώτημά σας. Επίσης, θα εμφανιστεί ένας σύνδεσμος που οδηγεί στο σημείο του βίντεο όπου βρίσκεται η απάντηση στην ερώτησή σας.

Δοκιμάστε μερικά από τα παρακάτω ερωτήματα:

- Τι είναι το Azure Machine Learning;
- Πώς λειτουργούν τα συνελικτικά νευρωνικά δίκτυα;
- Τι είναι ένα νευρωνικό δίκτυο;
- Μπορώ να χρησιμοποιήσω Jupyter Notebooks με το Azure Machine Learning;
- Τι είναι το ONNX;


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθεια για ακρίβεια, παρακαλούμε να γνωρίζετε ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρανοήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
